In [5]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import csv
import datetime as dt

In [6]:
# school
# hostip = "10.192.40.238"
hostip = "10.192.13.255"

# home
# hostip = "192.168.0.88"
# hostip = "192.168.0.160"
# 
# hostip = "10.192.12.188"
# hostip = "10.192.41.222"

# hostip = "192.168.0.21"

topic = 'ProducerB'
file_path = 'data/camera_event_B.csv' 
# file_path = 'A2/data/camera_event_B.csv' 



def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(data, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        print('Message published successfully. ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
        

In [7]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [8]:
if __name__ == '__main__':
    print('Publishing records..')
    producer = connect_kafka_producer()

    if producer is None:
        print('Failed to connect to Kafka. Exiting..')
        exit(1)

    current_batch = None
    batch_events = []

    with open(file_path, mode='r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            json_row = dumps(row)
            if current_batch is None:
                current_batch = row['batch_id']

            if current_batch != row['batch_id']:
                for event in batch_events:
                    message = {
                        'event_id': event['event_id'],
                        "car_plate": event['car_plate'],
                        "camera_id": int(event['camera_id']),
                        "timestamp": event['timestamp'],
                        "speed_reading": float(event['speed_reading']),
                        "producer": "B"
                    }
                    publish_message(producer, topic, event['event_id'], dumps(message))
                producer.flush()
                print(f'Batch {current_batch} published successfully.')
                sleep(5)

                current_batch = row['batch_id']
                batch_events = []

            batch_events.append(row)

        if batch_events:
            for event in batch_events:
                message = {
                    'event_id': event['event_id'],
                    "car_plate": event['car_plate'],
                    "camera_id": int(event['camera_id']),
                    "timestamp": event['timestamp'],
                    "speed_reading": float(event['speed_reading']),
                    "producer": "B"
                }
                publish_message(producer, topic, event['event_id'], dumps(message))
            producer.flush()
            print(f'Final batch {current_batch} published successfully.')

    print('All messages published successfully.')

Publishing records..
Message published successfully. {"event_id": "b5b7dae7-3bf9-4f75-aea8-398d3bde3a41", "car_plate": "CJW 924", "camera_id": 2, "timestamp": "2024-01-01T08:00:26.029536", "speed_reading": 156.8, "producer": "B"}
Message published successfully. {"event_id": "1649c03b-cd6f-47bd-b4f2-c44fcf60524d", "car_plate": "CZ 592", "camera_id": 2, "timestamp": "2024-01-01T08:00:26.729115", "speed_reading": 163.9, "producer": "B"}
Message published successfully. {"event_id": "a9c6a4b7-a6b6-4bd7-9719-88bd895814f4", "car_plate": "UTT 229", "camera_id": 2, "timestamp": "2024-01-01T08:00:26.887170", "speed_reading": 134.1, "producer": "B"}
Message published successfully. {"event_id": "4e13ec51-02db-442c-b7aa-3a2050dd1ad1", "car_plate": "WA 0712", "camera_id": 2, "timestamp": "2024-01-01T08:00:27.601617", "speed_reading": 133.8, "producer": "B"}
Message published successfully. {"event_id": "74248dd9-c9f8-48d2-94fb-dc5610a0f735", "car_plate": "AH 8", "camera_id": 2, "timestamp": "2024-01-

Message published successfully. {"event_id": "2ae298d2-5496-4732-95ef-82f4b57ef4b7", "car_plate": "KWO 421", "camera_id": 2, "timestamp": "2024-01-01T08:13:44.248324", "speed_reading": 115.4, "producer": "B"}
Message published successfully. {"event_id": "61ffc37e-73c0-4fa9-a604-e2596e8f6289", "car_plate": "ZV 2619", "camera_id": 2, "timestamp": "2024-01-01T08:13:48.703774", "speed_reading": 96.8, "producer": "B"}
Batch 20 published successfully.
Message published successfully. {"event_id": "b8bea07f-235c-4ce0-9b27-fb2f524fb331", "car_plate": "VKZ 5377", "camera_id": 2, "timestamp": "2024-01-01T08:13:57.545422", "speed_reading": 75.9, "producer": "B"}
Batch 21 published successfully.
Message published successfully. {"event_id": "3d5110f0-2eb0-4ff6-8319-53677a95fb5c", "car_plate": "DWT 789", "camera_id": 2, "timestamp": "2024-01-01T08:08:34.407062", "speed_reading": 135.4, "producer": "B"}
Message published successfully. {"event_id": "a5cea7f7-ffa2-497d-9ae2-305343afa884", "car_plate": "

Message published successfully. {"event_id": "25f89334-492a-43a8-9149-7e48566dff38", "car_plate": "AFS 0", "camera_id": 2, "timestamp": "2024-01-01T08:25:46.040096", "speed_reading": 126.8, "producer": "B"}
Message published successfully. {"event_id": "9fcd88c7-0936-4d2f-a1fc-b84626615008", "car_plate": "ZLY 15", "camera_id": 2, "timestamp": "2024-01-01T08:25:46.822689", "speed_reading": 150.5, "producer": "B"}
Message published successfully. {"event_id": "2378fdc8-efb2-4053-873e-9216523f8f06", "car_plate": "JYA 82", "camera_id": 2, "timestamp": "2024-01-01T08:25:47.987095", "speed_reading": 145.0, "producer": "B"}
Message published successfully. {"event_id": "4083acd5-d65b-4ef7-a122-8f277d57635d", "car_plate": "DGT 19", "camera_id": 2, "timestamp": "2024-01-01T08:25:49.140729", "speed_reading": 126.5, "producer": "B"}
Batch 42 published successfully.
Message published successfully. {"event_id": "988064e2-5e43-434b-868a-c201ca32240a", "car_plate": "TAD 5898", "camera_id": 2, "timestamp

Message published successfully. {"event_id": "5dce6c9d-79dd-4b9e-9a74-0401e305c707", "car_plate": "YTE 8888", "camera_id": 2, "timestamp": "2024-01-01T08:43:18.777511", "speed_reading": 93.8, "producer": "B"}
Message published successfully. {"event_id": "22889716-c948-4d08-9d49-6275deea693e", "car_plate": "CU 131", "camera_id": 2, "timestamp": "2024-01-01T08:43:21.733149", "speed_reading": 79.4, "producer": "B"}
Batch 58 published successfully.
Message published successfully. {"event_id": "d0fbaadb-ea27-418c-acf0-c68e0ce8058b", "car_plate": "OH 2", "camera_id": 2, "timestamp": "2024-01-01T08:43:24.544815", "speed_reading": 70.5, "producer": "B"}
Message published successfully. {"event_id": "b247b540-f113-4120-97c8-7c05899c4646", "car_plate": "BPV 0250", "camera_id": 2, "timestamp": "2024-01-01T08:43:25.555619", "speed_reading": 79.0, "producer": "B"}
Batch 59 published successfully.
Message published successfully. {"event_id": "8e6bbfa7-9d6d-49ca-8db9-c84eb9e4a9b3", "car_plate": "MC 27

KeyboardInterrupt: 